<a href="https://colab.research.google.com/github/mikael-daniels/team11_pipeline/blob/master/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building an ETL Pipeline

As the second part of the predict for Gather, you will need to build a pipeline of functions in python which does the following:

1. Function to connect to twitter and scrapes "Eskom_SA" tweets.
<br>
<br>
2. Cleans/Processes the tweets from the scraped tweets which will create a dataframe with two new columns using the following functions: <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a) Hashtag Remover from Analyse Functions
<br>
<br>
3. Functions which connects to your SQL database and uploads the tweets into the table you store the tweets in the database.

In [0]:
# General:
import tweepy           # To consume Twitter's API
import pandas as pd     # To handle data
import numpy as np      # For numerical computation
import json
# For plotting and visualization:
from IPython.display import display
import pyodbc


# Consumer and Access details

Fill in your Consumer and Access details you should have recieved when applying for a Twitter API. 

In [0]:
# Consumer:
CONSUMER_KEY    = 'PDU7fauXIq4hZRdkJ62s3OYPT'
CONSUMER_SECRET = 'D118AztFQ9y6irrMOKLKxuU6XLKiYznIZ5d93B4ZnO5SaAzHsB'

# Access:
ACCESS_TOKEN  = '2954682544-Ttwu770muNrhg4KQSY6N9RYGuK33Xs2V80q8Ms8'
ACCESS_SECRET = '95idH7IMmvwO74eOwnDGVasxGZ4E8ftEoGEYPhfGWOmKh'

In [0]:
# API's setup:
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with access and consumer keys from Twitter.
    """

    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth, timeout=1000)
    return api

# Function 1:

Write a function which:
- Scrapes _"Eskom_SA"_ tweets from Twitter. 

Function Specifications:
- The function should return a dataframe with the scraped tweets with just the "_Tweets_" and "_Date_". 
- Will take in the ```consumer key,  consumer secret code, access token``` and ```access secret code```.

NOTE:
The dataframe should have the same column names as those in your SQL Database table where you store the tweets.

In [0]:
def twitter_df(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET ):
    
    ''' Return a new dataframe with data scraped from Twitter
        
        Args: 'CONSUMER_KEY', 'CONSUMER_SECRET', 'ACCESS_TOKEN', 'ACCESS_SECRET'

        Returns: Dataframe with two columns: 'Tweets' and 'Date'
    ''' 
    # Code Here
    
    extractor = twitter_setup()
    
    # We create a tweet list as follows:
    
    tweets = extractor.user_timeline(screen_name="Eskom_SA", 
                                     count=100,
                                     include_rts=False)
    
    print(f"Number of tweets extracted: {len(tweets)}.\n")
    
    # We create a pandas dataframe as follows:
    
    data = pd.DataFrame(data=np.column_stack([[tweet.text for tweet in tweets],
                                              [(tweet.created_at) for tweet in tweets]]),
                                              columns=['Tweets','Date'])
    
    return data

# Function 2: Removing hashtags and the municipalities

Write a function which:
- Uses the function you wrote in the Analyse section to extract the hashtags and municipalities into it's own column in a new data frame. 

Function Specifications:
- The function should take in the pandas dataframe you created in Function 1 and return a new pandas dataframe. 

In [0]:
def extract_municipality_hashtags(df):
    
    ''' Return a dataframe with hashtags and municipalities extracted, using dataframe obtained from Function 1

        Args: 'df', Dataframe created by Function 1

        Returns: Original dataframe, 'df', with two new columns, 'hashtags' and 'municipality'
    '''
    mun_dict = {'@CityofCTAlerts' : 'Cape Town',
                '@CityPowerJhb' : 'Johannesburg',
                '@eThekwiniM' : 'eThekwini' ,
                '@EMMInfo' : 'Ekurhuleni',
                '@centlecutility' : 'Mangaung',
                '@NMBmunicipality' : 'Nelson Mandela Bay',
                '@CityTshwane' : 'Tshwane'}
    
    handles = list(mun_dict.keys())
    
    df['municipality'] = df['Tweets'].str.extract('({})'.format('|'.join(handles)), expand=False).fillna(np.nan)
    df['municipality'] = df['municipality'].map(mun_dict)
    
    df['hashtags'] = df['Tweets'].str.findall(r'#.*?(?=\s|$)')
    df['hashtags'] = df['hashtags'].apply(lambda x: np.nan if len(x)==0 else [x.lower() for x in x])
    

    return df

# Function 3: Updating SQL Database with pyODBC

Write a function which:
- Connects and updates your SQL database. 

Function Specifications:
- The function should take in a pandas dataframe created in Function 2. 
- Connect to your SQL database.
- Update the table you store your tweets in.
- Not return any output.

In [0]:
def pyodbc_twitter(connection, df, twitter_table):

    ''' Update 'Tweets' table in our SQL database

      Args: 'connection', 'df', 'twitter_table'

      Returns: None, only updates the 'Twitter' table
    '''

    ### Code Here
    #make a cursor object using connection construct
    cursor = connection.cursor()

    #convert the dataframe to .csv file    
    df.to_csv (r'C:\Users\monicafar147\Desktop\export_dataframe.csv', index = False, header=True)
    
    print('reading data from Tweets table in SQL:')    
    tweets = pd.read_sql_query('select * from dbo.Tweets',connection)
    display(tweets)
    
   
    return None

In [0]:
connection = pyodbc.connect(driver='{SQL Server}',
                      host='EDSA-3GKPSM2\SQLEXPRESSMON',
                      database='New Clean Database Gather',
                      trusted_connection='tcon',
                      user='sa')

In [0]:
twitter_table = twitter_df(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET )

Number of tweets extracted: 89.



In [0]:
new_df = extract_municipality_hashtags(twitter_table)
new_df.head()

,Tweets,Date,municipality,hashtags
0,#POWERALERT 1\n\nDate: 11 March 2020\n\nStage ...,2020-03-11 09:58:19,NaN,[#poweralert]
1,We are currently loadshedding in stage 4. \n\...,2020-03-11 07:42:17,NaN,NaN
...,...,...,...,...
3,Eskom is currently loadshedding in stage 2\n\n...,2020-03-11 03:39:22,NaN,NaN
4,#POWERALERT 3\nDate: 10 March 2020\n\nStage 4 ...,2020-03-10 16:42:19,NaN,[#poweralert]


In [0]:
pyodbc_twitter(connection, new_df, twitter_table)

reading data from Tweets table in SQL:


,tweets,dates,municipality,hashtags
0,#POWERALERT 1\n\nDate: 11 March 2020\n\nStage ...,2020-03-11 09:58:19,None,['#poweralert']
1,We are currently loadshedding in stage 4. \n\...,2020-03-11 07:42:17,None,None
...,...,...,...,...
87,#POWERALERT 1\nDate: 03 March 2020\n\nNo loads...,2020-03-03 06:30:54,None,['#poweralert']
88,Prepaid pricing is based on usage/ consumption...,2020-03-03 06:30:00,None,None


# Open SQLExpress and run the following queries
- drop table Tweets

- create table Tweets(tweets text,dates text, municipality text, hashtags text);

- BULK INSERT Tweets
FROM 'C:\Users\monicafar147\Desktop\export_dataframe.csv'
WITH ( FORMAT='CSV');

- select * from Tweets

In [0]:
pyodbc_twitter(connection, new_df, twitter_table)

reading data from Tweets table in SQL:


,tweets,dates,municipality,hashtags
0,#POWERALERT 1\n\nDate: 11 March 2020\n\nStage ...,2020-03-11 09:58:19,None,['#poweralert']
1,We are currently loadshedding in stage 4. \n\...,2020-03-11 07:42:17,None,None
...,...,...,...,...
87,#POWERALERT 1\nDate: 03 March 2020\n\nNo loads...,2020-03-03 06:30:54,None,['#poweralert']
88,Prepaid pricing is based on usage/ consumption...,2020-03-03 06:30:00,None,None
